In [ ]:
# DRO analysis

In [ ]:
import numpy as np, pandas as pd, 
import matplotlib.pyplot as plt
import pickle, gzip, os

In [ ]:
# Import DRO results
out = load_out("/Output/dro_run.pkl.gz")

In [ ]:
# Print results
results = {
    "MVO":          pd.DataFrame([out["MVO"]["summary"]]),
    "DRO (Static)": pd.DataFrame([out["PartA"]["summary"]]),
    "Regime-DRO":   pd.DataFrame([out["PartB"]["summary"]]),
}

tbl = oos_summary(results, model_order=["MVO","DRO (Static)","Regime-DRO"])
display(tbl.loc[["mu_ann","sigma_ann","sharpe_ann","vol_breach","max_drawdown"]])

In [ ]:
# Plot Portfolio A vs Portfolio B

# pull series safely
sA = pd.Series(out["series"].get("PartA_daily", pd.Series(dtype=float))).sort_index()
sB = pd.Series(out["series"].get("PartB_daily", pd.Series(dtype=float))).sort_index()

if sA.empty or sB.empty:
    raise ValueError("Missing series in `out['series']`. Expected 'PartA_daily' and 'PartB_daily'.")

# cumulative returns
cumA = (1 + sA).cumprod() - 1
cumB = (1 + sB).cumprod() - 1

# --- cumulative on one chart ---
fig, ax = plt.subplots(figsize=(8, 4))
cumA.plot(ax=ax, label="Portfolio A (Static)")
cumB.plot(ax=ax, label="Portfolio B (Regime)")
ax.set_title("Cumulative Return")
ax.set_xlabel("Date")
ax.set_ylabel("Cumulative return")
ax.grid(True, alpha=0.3)
ax.legend()

In [ ]:
# Show deltas got tighter

summA = pd.Series(out["PartA"]["summary"])
summB = pd.Series(out["PartB"]["summary"])

# pull delta and per-segment deltas
delta_A = float(summA.get("delta", float("nan")))
delta_ks = [summB[k] for k in summB.index if str(k).startswith("delta_k")]
display(pd.DataFrame({
    "Static δ (Part A)": [delta_A],
    "Regime δ_k (Part B)": [pd.Series(delta_ks, dtype=float).describe()[["count","mean","min","25%","50%","75%","max"]].to_dict()]
}))


In [ ]:
# Compare OOS performance

from dro import oos_summary

results = {
    "DRO (Static)": pd.DataFrame([out["PartA"]["summary"]]),
    "Regime-DRO":   pd.DataFrame([out["PartB"]["summary"]]),
}
tbl = oos_summary(results, model_order=["DRO (Static)","Regime-DRO"])
display(tbl.loc[["mu_ann","sigma_ann","sharpe_ann","vol_breach","max_drawdown"]])


In [ ]:
# Quick hypothesis checks (paired tests work with m=1; p’s will be NaN—ok for a single trial; for multiple trials, they’ll be informative)

from dro import hypothesis_tests

hypothesis_tests(
    results,
    tests=[
        {"kind":"breach_less",        "A":"Regime-DRO", "B":"DRO (Static)"},
        {"kind":"superiority_sharpe", "A":"Regime-DRO", "B":"DRO (Static)"},
    ],
    alpha=0.05,
)

# Optional: add MVO baseline with fit_mvo(...) exactly like Part-A (set delta=0) and append to results.

In [ ]:
# Confirm:
# Expecting to see: the distribution of delta_k in Part B is typically below Part A. When regimes are stationary → tighter radii.
# OOS: higher Sharpe / lower breach and often lower drawdown for Regime-DRO vs static DRO (and vs MVO), on the same risk budget.